In [1]:
from keplergl import KeplerGl
import numpy as np
import pandas as pd
import geopandas as gpd
import glob
import os
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact

In [2]:
# Caminhos para os shapefiles
shapefile_path = '804387637e9638887b3bb625354623c5c83526532fac778ce885584328e169e4-1719798137525.shp' 
subpref_shapefile_path = 'distrito/distrito.shp' 

# Carregar o shapefile das arestas
gdf_edges = gpd.read_file(shapefile_path)

# Carregar o shapefile dos distritos
subpref_gdf = gpd.read_file(subpref_shapefile_path)
subpref_gdf = subpref_gdf.set_crs(epsg=31983)
subpref_names = subpref_gdf['ds_subpref'].unique().tolist()

# Garantir que os CRS são iguais
gdf_edges = gdf_edges.to_crs(subpref_gdf.crs)

# Carregar e processar os dados CSV
caminho_pasta = "csvs"
arquivos_csv = glob.glob(os.path.join(caminho_pasta, "*.csv"))

lista_dataframes = []
for arquivo in arquivos_csv:
    temp_df = pd.read_csv(arquivo, parse_dates=['hour'])
    lista_dataframes.append(temp_df)

df_all = pd.concat(lista_dataframes, ignore_index=True)  

In [3]:
# Extrair componentes de data e hora
df_all['date'] = pd.to_datetime(df_all['hour'])
df_all['year'] = df_all['date'].dt.year
df_all['month'] = df_all['date'].dt.month
df_all['day_of_week'] = df_all['date'].dt.dayofweek
df_all['hour_of_day'] = df_all['date'].dt.hour

# Calcular os totais para gêneros e faixas etárias
df_all['male_people_count'] = df_all['forward_male_people_count'].fillna(0) + df_all['reverse_male_people_count'].fillna(0)
df_all['female_people_count'] = df_all['forward_female_people_count'].fillna(0) + df_all['reverse_female_people_count'].fillna(0)
df_all['unspecified_people_count'] = df_all['forward_unspecified_people_count'].fillna(0) + df_all['reverse_unspecified_people_count'].fillna(0)

df_all['18_34_people_count'] = df_all['forward_18_34_people_count'].fillna(0) + df_all['reverse_18_34_people_count'].fillna(0)
df_all['35_54_people_count'] = df_all['forward_35_54_people_count'].fillna(0) + df_all['reverse_35_54_people_count'].fillna(0)
df_all['55_64_people_count'] = df_all['forward_55_64_people_count'].fillna(0) + df_all['reverse_55_64_people_count'].fillna(0)
df_all['65_plus_people_count'] = df_all['forward_65_plus_people_count'].fillna(0) + df_all['reverse_65_plus_people_count'].fillna(0)

df_all['commute_trip_count'] = df_all['forward_commute_trip_count'].fillna(0) + df_all['reverse_commute_trip_count'].fillna(0)
df_all['leisure_trip_count'] = df_all['forward_leisure_trip_count'].fillna(0) + df_all['reverse_leisure_trip_count'].fillna(0)
df_all['total_trip_count'] = df_all['commute_trip_count'] + df_all['leisure_trip_count']

# Mapear números dos meses para nomes em português
month_names = {
    1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 4: 'Abril', 5: 'Maio', 6: 'Junho',
    7: 'Julho', 8: 'Agosto', 9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'
}

In [ ]:
# Criar widgets padronizados
subpref_widget = widgets.Dropdown(
    options=subpref_names,
    description='Distrito:',
    disabled=False,
)

activity_type_widget = widgets.Dropdown(
    options=['Todos', 'Deslocamento', 'Lazer'],
    value='Todos',
    description='Tipo de Atividade:'
)

ano_widget = widgets.SelectMultiple(
    options=[2019, 2020, 2021, 2022, 2023, 2024],
    value=[2019, 2020, 2021, 2022, 2023, 2024],
    description='Ano:'
)

mes_widget = widgets.SelectMultiple(
    options=[('Janeiro', 1), ('Fevereiro', 2), ('Março', 3), ('Abril', 4), ('Maio', 5), ('Junho', 6),
             ('Julho', 7), ('Agosto', 8), ('Setembro', 9), ('Outubro', 10), ('Novembro', 11), ('Dezembro', 12)],
    value=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    description='Mês:'
)

genero_widget = widgets.Dropdown(
    options=['Todos', 'Homem', 'Mulher', 'Não especificado'],
    value='Todos',
    description='Gênero:'
)

faixa_etaria_widget = widgets.Dropdown(
    options=['Todos', '18-34', '35-54', '55-64', '65+'],
    value='Todos',
    description='Faixa Etária:'
)

# Dicionários de mapeamento
map_genero = {
    'Todos': 'total',
    'Homem': 'male',
    'Mulher': 'female',
    'Não especificado': 'unspecified'
}

map_atividade = {
    'Todos': 'total',
    'Deslocamento': 'commute',
    'Lazer': 'leisure'
}

map_faixa_etaria = {
    'Todos': 'total',
    '18-34': '18_34',
    '35-54': '35_54',
    '55-64': '55_64',
    '65+': '65_plus'
}

Aplicar Filtros

In [5]:
def update_df(ano, mes, genero, faixa_etaria, activity_type):
    df_filtered = df_all.copy()
    
    # Filtrar pelos anos e meses selecionados
    df_filtered = df_filtered[df_filtered['year'].isin(ano)]
    df_filtered = df_filtered[df_filtered['month'].isin(mes)]
    
    if df_filtered.empty:
        print("Não há dados disponíveis para os filtros selecionados.")
        return pd.DataFrame()
    
    # Mapear as opções selecionadas para os valores correspondentes
    activity_type_mapped = map_atividade[activity_type]
    genero_mapped = map_genero[genero]
    faixa_etaria_mapped = map_faixa_etaria[faixa_etaria]
    
    # Selecionar a coluna de contagem de viagens apropriada
    if activity_type_mapped == 'total':
        trip_count_column = 'total_trip_count'
    else:
        trip_count_column = activity_type_mapped + '_trip_count'
    
    # Tratar a seleção de gênero
    if genero_mapped != 'total':
        df_filtered['people_count'] = df_filtered[genero_mapped + '_people_count'].fillna(0)
    else:
        df_filtered['people_count'] = df_filtered[['male_people_count', 'female_people_count', 'unspecified_people_count']].fillna(0).sum(axis=1)
    
    # Tratar a seleção de faixa etária
    if faixa_etaria_mapped != 'total':
        df_filtered['age_group_count'] = df_filtered[faixa_etaria_mapped + '_people_count'].fillna(0)
    else:
        df_filtered['age_group_count'] = df_filtered[['18_34_people_count', '35_54_people_count', '55_64_people_count', '65_plus_people_count']].fillna(0).sum(axis=1)
    
    # Filtrar dados onde as contagens de pessoas e faixa etária são maiores que zero
    df_filtered = df_filtered[(df_filtered['people_count'] > 0) & (df_filtered['age_group_count'] > 0)]
    
    if df_filtered.empty:
        print("Não há dados disponíveis para os filtros selecionados.")
        return pd.DataFrame()
    
    # Agrupar os dados por 'edge_uid' e somar as contagens de viagens
    df_grouped = df_filtered.groupby('edge_uid', as_index=False)[trip_count_column].sum()
    df_grouped['edge_uid'] = df_grouped['edge_uid'].astype('int32')
    df_grouped.rename(columns={trip_count_column: 'total_trip_count'}, inplace=True)
    
    return df_grouped

In [ ]:
def plot_map(df_grouped, selected_subpref, activity_type, genero, faixa_etaria, ano, mes):
    if df_grouped.empty:
        return
    # Filtrar o distrito selecionado
    selected_subpref_gdf = subpref_gdf[subpref_gdf['ds_subpref'] == selected_subpref]
    
    # Mesclar os dados agregados com o GeoDataFrame das arestas
    merged_gdf = gdf_edges.merge(df_grouped, left_on='edgeUID', right_on='edge_uid')

    merged_gdf['log_total_trip_count'] = np.log1p(merged_gdf['total_trip_count'])
    # Realizar interseção espacial para obter apenas as arestas dentro do distrito selecionado
    gdf = gpd.sjoin(merged_gdf, selected_subpref_gdf, how='inner', predicate='intersects')
    
    gdf = gdf.to_crs(epsg=4326)
    selected_subpref_gdf = selected_subpref_gdf.to_crs(epsg=4326)
    
    bounds = selected_subpref_gdf.total_bounds  
    minx, miny, maxx, maxy = bounds
    center_lat = (miny + maxy) / 2
    center_lon = (minx + maxx) / 2

    # Criar o mapa KeplerGl
    kepler_map = KeplerGl(height=600)

    # Adicionar dados ao mapa
    kepler_map.add_data(data=gdf, name='Trip Data')

    # Configuração do mapa
    config = {
        "version": "v1",
        "config": {
            "visState": {
                "layers": [
                    {
                        "id": "Trip Data",
                        "type": "geojson",
                        "config": {
                            "dataId": "Trip Data",  
                            "label": "Trip Data",
                            "color": [255, 203, 153],
                            "columns": {
                                "geojson": "geometry"
                            },
                            "isVisible": True,
                            "visConfig": {
                                "opacity": 0.8,
                                "strokeOpacity": 1,
                                "thickness": 0.5,
                                "colorRange": {
                                    "name": "Global Warming",
                                    "type": "sequential",
                                    "category": "Uber",
                                    "colors": [
                                        "#5A1846",
                                        "#900C3F",
                                        "#C70039",
                                        "#E3611C",
                                        "#F1920E",
                                        "#FFC300"
                                    ]
                                },
                                "strokeColorRange": {
                                    "name": "ColorBrewer OrRd-9",
                                    "type": "sequential",
                                    "category": "ColorBrewer",
                                    "colors": [
                                        "#7f0000",
                                        "#b30000",
                                        "#d7301f",
                                        "#ef6548",
                                        "#fc8d59",
                                        "#fdbb84",
                                        "#fdd49e",
                                        "#fee8c8",
                                        "#fff7ec"
                                    ],
                                    "reversed": False
                                },
                                "stroked": True,
                                "filled": False,
                                "enable3d": False
                            },
                            "hidden": False,
                            "textLabel": [
                                {
                                    "field": None,
                                    "color": [255, 255, 255],
                                    "size": 18,
                                    "offset": [0, 0],
                                    "anchor": "start",
                                    "alignment": "center"
                                }
                            ]
                        },
                        "visualChannels": {
                            "strokeColorField": {
                                "name": "log_total_trip_count",
                                "type": "real"
                            },
                            "strokeColorScale": "quantize"
                        }
                    }
                ],
                "interactionConfig": {
                    "tooltip": {
                        "fieldsToShow": {
                            "Trip Data": [  
                                {"name": "edgeUID", "format": None},
                                {"name": "total_trip_count", "format": None}
                            ]
                        },
                        "enabled": True
                    }
                },
                "layerBlending": "normal",
                "overlayBlending": "normal"
            },
            "mapState": {
                "bearing": 0,
                "dragRotate": False,
                "latitude": center_lat,
                "longitude": center_lon,
                "pitch": 0,
                "zoom": 12,
                "isViewportSynced": True
            },
            "mapStyle": {
                "styleType": "light",
                "visibleLayerGroups": {
                    "label": True,
                    "road": True,
                    "border": False,
                    "building": True,
                    "water": True,
                    "land": True,
                    "3d building": False
                },
                "threeDBuildingColor": [15.035172933000911, 15.035172933000911, 15.035172933000911],
                "backgroundColor": [0, 0, 0]
            }
        }
    }

    # Aplicar a configuração
    kepler_map.config = config

    output_dir = 'map-output'
    os.makedirs(output_dir, exist_ok=True)

    # Construir o nome do arquivo
    ano_str = f"{min(ano)}-{max(ano)}" if len(ano) > 1 else str(ano[0])
    mes_str = f"{min(mes)}-{max(mes)}" if len(mes) > 1 else str(mes[0])
    file_name = f"mapa_subpref_{selected_subpref}_atividade_{activity_type}_genero_{genero}_faixa_etaria_{faixa_etaria}_ano_{ano_str}_mes_{mes_str}.html"
    file_name = file_name.replace(' ', '_').replace(',', '')  
    file_path = os.path.join(output_dir, file_name)
    
    # Salvar o mapa em HTML com o caminho do arquivo construído
    kepler_map.save_to_html(file_name=file_path)
    
    # Exibir o mapa
    display(kepler_map)

In [ ]:
def interactive_plot(selected_subpref, activity_type, genero, faixa_etaria, ano, mes):
    df_grouped = update_df(ano, mes, genero, faixa_etaria, activity_type)
    plot_map(df_grouped, selected_subpref, activity_type, genero, faixa_etaria, ano, mes)

interact(interactive_plot, selected_subpref=subpref_widget, activity_type=activity_type_widget, genero=genero_widget, faixa_etaria=faixa_etaria_widget, ano=ano_widget, mes=mes_widget)

interactive(children=(Dropdown(description='Distrito:', options=('SANTANA-TUCURUVI', 'PARELHEIROS', 'VILA MARI…

<function __main__.interactive_plot(selected_subpref, activity_type, genero, faixa_etaria, ano, mes)>